In [1]:
import pandas as pd
import numpy as np
import pandasql as sqldf
import datetime
import matplotlib.pyplot as plt
import sys,os
from sklearn.model_selection import train_test_split

**Einlesen und abspeichern:**

In [3]:
csv_filenames = []
#auslesen aller csv file dateinamen aus formula 1 datensatz und abspeichern in liste

for filename in os.listdir(os.getcwd()+'/kaggle_data'):
    typ = filename.split('.')[-1]
    name = filename.split('.')[0]
    if typ == 'csv':
        csv_filenames.append(filename)

In [6]:
#Speichern aller CSV Dateinen im Dict "all_dfs" mit deren Namen als key
all_dfs = {}
#einlesen und abspeichern als dataframe aller dateien
for file in csv_filenames:
    #print(file) #Um anzuzeigen, welche Datei genau eingelesen wird
    path = 'kaggle_data/'+file
    try:
        df = pd.read_csv(path, engine = 'python', sep = ',')
    except Exception as e:
        df = pd.read_csv(path, engine = 'c', sep = ',') #Da ein Null byte in qualifying und races enthalten ist,
                                                        #kann die python Engine das so nicht verarbeiten
        #print(e) #Gibt Error aus, der Entstanden ist
    #print(df.head())
    all_dfs[file] = df

**Mergen der Daten verschiedener Dateien**

In [7]:
#Zur Übersichtlichkeit übertragen der DFs in Distincte Variablen
laps = all_dfs['lapTimes.csv']
drivers = all_dfs['drivers.csv']
races = all_dfs['races.csv']
result = all_dfs['results.csv']
constructor = all_dfs['constructors.csv']
pits = all_dfs["pitStops.csv"]
status = all_dfs["status.csv"]

In [28]:
"""Relevante Dataframes werden zusammengemerged und nur Informationen über 'raceId','year', 'circuitId',
'driverId','lap_number', 'lap_position', 'lap_in_milliseconds', werden behalten, um ein 
Übersichlicheres Dataframe zu generieren. Aufbau mithilfe von SQL recht variabel, welche Infos mitzunehmen sind """


merged_df = sqldf.sqldf("""select 
            t1.raceId,
            t1.year,
            --t1.round as race_round_number,
            t1.circuitId, 
            --t1.name as grandprix_name,
            --t1.date,
            --t1.time as race_time,
            --t2.round as round_number,
            t2.driverId,
            t2.lap as lap_number,
            t2.position as lap_position,
            --t2.time as lap_time,
            t2.milliseconds as lap_in_milliseconds,
            --t2.driverRef,
            --t2.number as driver_number,
            --t2.code as driver_code,
            --t2.forename,
            --t2.surname,
            t2.driver_fullname
            --t2.dob as driver_dob,
            --t2.nationality as driver_nationality
            from 
            races t1 join (select 
                d1.*,
                d2.driverRef,
                d2.number,
                d2.code,
                d2.forename,
                d2.surname,
                d2.forename||' '||d2.surname as driver_fullname,
                d2.dob,
                d2.nationality
               -- d2.url
                from laps d1 join drivers d2
                on d1.driverId=d2.driverId) t2
            on t1.raceId=t2.raceId
            order by t1.raceId
            """)



Output der obrigen Zelle:
DataFrame mit folgenden Informationen:
    1) raceId              --kann jedes gafahrene Rennen eindeutig identifizieren
    2) year                --enthält das Jahr, in dem das Rennen gefahren wurde
    3) circuitId           --Streckennummer (eindeutige ID)
    4) driverId            --IDs der Fahrer
    5) lap_number          --Rundennummer (pro Runde im DataFrame ein Eintrag)
    6) lap_position        --Position des Fahrers in der aktuell gefahrenen Runde
    7) lap_in_milliseconds --Dauer, die der Fahrer für die eine Runde gebraucht hat

In [29]:
#Hinzufügen der constuctorId (=Eindeutige Kennzeichnung eines Teams) und der späteren Endposition
merged_with_conpos = pd.merge(merged_df,result[['raceId', 'driverId','position', 'constructorId']], on=['raceId','driverId']) 
merged_with_conpos.rename(columns = {'position':'podium_position'}, inplace = True)  #Rename für Eindeutigere Bezeichnung 

**Pitstops**

In [30]:
#Information hinzufügen, in welcher Runde ein Fahrer gestoppt hat
pits["stop_binary"] = 1 #Neue Spalte erstellen, die angbit, ob ein Fahrer in einer bestimmten Runde gestoppt hat (Box)
pits.rename(columns = {"lap": "lap_number"}, inplace = True) #Rename der Spalte lap in der Pits Tabelle zu lap_number um merge leichter durchzuführen
#Merge Durchführen, um auch Pits in merged_with_conpos zu bekommen
merged_with_conpos = pd.merge(merged_with_conpos, pits[['raceId', 'driverId', 'lap_number', 'stop_binary']], on = ['raceId','driverId', 'lap_number'], how = "outer")
#Nur die Rennen, ab 2011 beachten, wegen großer Umstellungen im Regelwerk (z.B. darf nichtmehr getankt werden und Einführung DRS)
fin_newage = merged_with_conpos.where(merged_with_conpos.year >= 2011).dropna(how = "all") 
fin_newage["stop_binary"] = fin_newage["stop_binary"].replace(np.nan, 0) #Nans ersetzen
#fin_newage.head(20)


**Max Runden**

In [31]:
#Herausfinden der Rundenanzahl, die pro Rennen gefahren werden muss und in DF zwischenspeichern
Max_rounds = sqldf.sqldf("""
                            select *, max(lap_number) as total_laps
                            from fin_newage group by raceId """)

#Mergen des eben erstellten DFs mit der Maximalen Rundenanzahl und dem aktuellen DF
fin_newage = pd.merge(fin_newage, Max_rounds[['raceId', 'total_laps']], on ='raceId', how = "outer")

#Berechen zu wie viel % ein Fahrer in einer gewissen Runde das Rennen schon hinter sich gebracht hat 
#Annahme: Jedes Rennen wird über die volle Distanz gefahren
fin_newage["race_completion"] = fin_newage["lap_number"]/fin_newage["total_laps"]

**Status**

In [32]:
#Den staus aus der .csv Datei mit dem Status herausnehmen und identifizieren
status = pd.merge(result, status[['statusId','status']], on ='statusId', how = "outer")
status_df = sqldf.sqldf(""" select * from status where raceId >= 841""") #Weil nur Rennen ab 2011 in beatracht gezogen werden

In [33]:
#anlegen einer temporären Spalte, die anzahl der überrundeten Runden enthält, bzw. sinnfrei zerschnittene strings
status_df["count_laps"] = status_df["status"].apply(lambda y: y.split()[0][1:])

def conv_int(x):
    #konvertieren des eintrags in den typ integer, um später vergleich vornehmen zu können
    try:
        x = int(x)
    except:
        #fall kommt nur vor wenn keine überrundung stattgefunden hat, also im generellen DNF fall oder Finished
        x = 1000
        
    return x
#anwenden der conv_int funktion auf die zuvor erzeugte temp spalte
status_df["count_laps"] = status_df.count_laps.apply(lambda y: conv_int(y))       

#hinzufügen der spalte sttatus_clean, mit nur drei Merkmalsausprägungen Finished, lapped und DNF
status_df["status_clean"] = ['No_DNF' if status_df.at[v,"status"] == 'Finished' else 'No_DNF' if (status_df.at[v,"status"].startswith('+') and status_df.at[v,"count_laps"] <= 3)else 'DNF' for v in range(len(status_df.status))]

#zusammenführen von status_df informationen und bisherigem dataframe
fin_newage = pd.merge(fin_newage, status_df[['raceId', 'driverId', 'grid', 'status_clean', 'count_laps']], on = ['raceId', 'driverId'])


#Status nur noch Binär Darstellen: 1 = No_DNF, 0 = DNF  --> Rest nutzlose Information
fin_newage['status_binary'] = [1 if fin_newage.at[v,'status_clean']== 'No_DNF' else 0 for v in range(len(fin_newage.status_clean))]
fin_newage.where(fin_newage.status_clean == 'DNF').dropna(how = 'all')
test = fin_newage.replace(np.nan, 'lul')
filt1 = test.podium_position != 'lul'
filt2 = test.status_clean == 'DNF'
#test.where(filt1 & filt2).dropna(how = 'all')

**Total_minutes**

In [34]:
#Generieren eines Temporären DFs, um die gesamt gefahrenen ms pro Fahrer pro Rennen zu berechnen
t = sqldf.sqldf("""select *, sum(lap_in_milliseconds) as total_milliseconds_temp
                from fin_newage
                group by raceId, driverId""")

t['total_milliseconds'] = [t.at[x, 'total_milliseconds_temp'] if t.at[x,'count_laps'] == 1000 else t.at[x, 'total_milliseconds_temp']+(t.at[x, 'total_milliseconds_temp']/(t.at[x, 'total_laps']- t.at[x, 'count_laps']))*t.at[x, 'count_laps'] for x in range(len(t))]


#######################
#######################




#Informationen Entsprechend zusammenführen und zwischenvariable rausschmeißen
fin_newage = pd.merge(fin_newage, t[['raceId', 'driverId', 'total_milliseconds']], on = ['raceId', 'driverId'], how = 'outer')
fin_newage["total_minutes"] = fin_newage['total_milliseconds']/60000  #ms in m umrechnen
del fin_newage["total_milliseconds"]
del fin_newage["count_laps"]

**Regenrennen identifizieren**

In [35]:
#Da Information über Regenrennen nicht bekannt sind im Datensatz, muss der Datensatz erweitert werden
#1 bei Regen 0 bei nicht Regen
fin_newage['rain'] = 0
for i in range(len(fin_newage)):
    if fin_newage.loc[i,'raceId'] in [847,861,879,910,914,934,942,953,957,967,950,982]:
        fin_newage.loc[i,"rain"] = 1

**Berechnen der Form für jeden Fahrer**
Form gibt eine Aussage darüber, wie gut der Fahrer in den vorherigen 3 Rennen der Saison abgeschnitten hat, desto geringer die Form des Fahreres, desto wahrscheinlicher ist es, dass ein Fahrer auch im nächsten Rennen gut abschneidet. Bei DNF wird für dieses Rennen die Startposition genommen +3 Plätze zusätzlich als Strafe.


In [36]:
years = fin_newage.year.tolist()
years = set(years) #Unique
form_df = pd.DataFrame(columns = ["raceId", "driverId", "form"])
letzter_platz = pd.DataFrame(columns = ["year", "letzter_nummer"])
a = 0

for year in years:
    #filtern nach einem speziellen Jahr
    temp_df = fin_newage.where(fin_newage.year == year).dropna(how = "all")
    #auslesen aller Teams *2 = Anzahl der Rennteilnehmer, also schlechteste Rennposition
    constructor_count = len(sqldf.sqldf("""select distinct(constructorId)
                                    from temp_df 
                                    group by constructorId"""))*2
    #zwischenspeichern der constructor count informationen
    zeile = pd.DataFrame(columns = letzter_platz.columns)
    zeile.loc[0, :] = [year, constructor_count]
    letzter_platz = letzter_platz.append(zeile)
    temp_drivers = set(temp_df.driverId.tolist())
    for driver in temp_drivers:
        #filtern nach einem speziellen Fahrer
        temp_df2 = temp_df.where(temp_df.driverId == driver).dropna(how = "all")
        temp_df2 = sqldf.sqldf("""select raceId, driverId, podium_position,status_clean, grid from temp_df2 group by raceId order by raceId""")
        status = list(temp_df2['status_clean'])
        if 'DNF'in status:
            #wenn DNF in Rennen wird podiums position auf grid +3 gesetzt
            temp_df2['podium_position'] = [temp_df2.at[v, 'podium_position'] if temp_df2.at[v,'status_clean'] == 'No_DNF' else (temp_df2.at[v,'grid']+3) for v in range(len(temp_df2))]
            
        #Ersetzen der Rennen in denen DNF, also NaN mit letztem Platz constructor_count
        #temp_df2.replace(np.nan, constructor_count, inplace = True)
        
        #liste mit allen rennids wird erzeugt, um zu überprüfen ob welche in Abschnitt fehlen
        start_race_id =  temp_df2.at[0, "raceId"]
        end_race_id = temp_df2.at[list(temp_df2.index)[-1], "raceId"]
        r_ids = [x for x in range(int(start_race_id), int(end_race_id)+1)]
        temp_ids = temp_df2.raceId.astype(int)
        temp_ids = temp_ids.values.tolist()
        
        #überprüfen ob alle raceIds in der Liste vorhanden sind
        if temp_ids != r_ids:
            #fehlende rennen werden in liste gespeichert
            missing = [x for x in r_ids if x not in temp_ids]
            
            for miss in missing:
                #einfügen der fehlenden raceids in den dataframe
                platzhalter = pd.DataFrame(columns = temp_df2[['raceId','driverId','podium_position']].columns)
                #dataframe an stelle wo id fehlt aufteilen in vorher und nachher
                vorher = temp_df2.where(temp_df2.raceId < miss).dropna(how = "all")
                nachher = temp_df2.where(temp_df2.raceId > miss).dropna(how = "all")
                #erzeugen einer neuen Reihe, die fehlende id, driverId und als pp den letztmöglichen Platz enthält
                d = vorher.at[list(vorher.index)[0], "driverId"]
                platzhalter.loc[0,:] = [miss, d, constructor_count]
                vorher = vorher.append(platzhalter)
                temp_df2 = vorher.append(nachher)
                temp_df2.reset_index(inplace = True, drop = True)
                
        for i, row in temp_df2.iterrows():
            #form des fahrers anhand seiner podiums positionen der in der saison vorhergegangenen rennen bestimmen
            #Tatsächliche berechnung
            raceId = row["raceId"]
            driverId = row["driverId"]
            #i = 4
            if i == 0:
                form = 0
            elif i < 4:
                zaehler = temp_df2.loc[:i-1,"podium_position"].tolist()
                form = np.sum(zaehler)/i
            else:
                zaehler = temp_df2.loc[i-4:i-1,"podium_position"].tolist()
                form = np.sum(zaehler)/4
                
            form_df.loc[a, :] = [raceId, driverId, form]
            #print("form df", form_df)
            a += 1

In [37]:
#Information über den Status in Fin_newage bringen 
final_df = pd.merge(fin_newage, form_df, on = ["raceId", "driverId"])
final_df.form = final_df.form.astype(float)

In [38]:
global letzter_platz
#ersetzen von nan in der podiums position mit dem letzten platz aus dem jeweiligen jahr (NICHT ZUR FORMBERECHNUNG)
def apply_pp (row):
    global letzter_platz
    pp = row["podium_position"]
    year = row["year"]
    if np.isnan(pp):
        df_ = letzter_platz.where(letzter_platz.year == year).dropna(how = "all")
        pp_neu = df_.loc[list(df_.index)[0],"letzter_nummer"]
        return pp_neu
    else:
        return pp
#ersetzen der NaN in podium position, wegen DNF, durch letzten Platz in dem betroffenen Rennen
#Aufruf der obrigen Fkt
final_df["podium_position"] = final_df.apply(apply_pp, axis = 1)

In [39]:
final_df.columns

Index(['raceId', 'year', 'circuitId', 'driverId', 'lap_number', 'lap_position',
       'lap_in_milliseconds', 'driver_fullname', 'podium_position',
       'constructorId', 'stop_binary', 'total_laps', 'race_completion', 'grid',
       'status_clean', 'status_binary', 'total_minutes', 'rain', 'form'],
      dtype='object')

**Nur das mitnehmen was auch gebraucht wird**

In [40]:
#Nur die Columns mitnehmen, die später gebraucht werden
df = pd.DataFrame()
df = final_df[['raceId','driverId','driver_fullname','year','podium_position','lap_in_milliseconds','status_clean','status_binary','lap_number','circuitId','lap_position','constructorId','stop_binary','race_completion','grid','form','rain','total_minutes']]
df['bias'] = 1 #Bias hinzufügen

**Dummie Variablen erstellen**

In [41]:
#Aus circuitId und constructorId Dummie Variablen machen, da diese Nominal sind
#Vor slicing, da beispielsweise nicht alle Teams in jedem Jahr mitgefahren sind,
#aber beispielsweise ein NN immer die selbe Input Size braucht
df = pd.get_dummies(df, columns=['circuitId', 'constructorId'])

**slicing auf 50% jedes einzelne Rennen**

In [42]:
#dictionary soll einzelne datensätze zu jedem rennen enthalten
split_by_race = {}
#grenzwert ab dem vorhersage über rennausgang gemacht werden soll (orientiert sich an race_completion)
border = 0.5 #nach 50% des Rennens soll der Ausgang vorhergesagt werden
#dictionary, welches renndatensätze nur bis zu einem gewissen zeitpunkt enthält (zp wird nach border gewählt)
sliced_races = {}

for rid in df['raceId'].unique():
    race = df.where(df.raceId == rid).dropna(how = 'all')
    race.reset_index(inplace = True, drop = True)
    split_by_race[rid] = race
    
    #finden der lap_number wo race_completion die angegebene border überschreitet
    last_lap_num = race.where(race.race_completion == min(race.where(race.race_completion >= border).dropna(how = 'all')['race_completion'])).dropna(how = 'all')['lap_number'].unique()[0]
    
    #es werden nur daten aus race genommen, die bis zu dieser lap_number gehen
    race_shortened = race.where(race.lap_number < last_lap_num).dropna(how = 'all')
    race_shortened.reset_index(inplace = True, drop = True)
    
    #hinzufügen einer spalte die die insgesamt gefahrenen millisekunden enthält bis zu dieser Runde pro Fahrer enthält
    race_shortened['sum_milliseconds_pro_lap'] = 0
    
    for did in race_shortened.driverId.unique():
        
        l_lap_driver = race_shortened.where(race_shortened.driverId == did).dropna(how = 'all')['lap_number'].tail(1).reset_index()['lap_number'][0]
        status_driver = race_shortened.status_clean.unique()[0]
        if status_driver == 'DNF':
            if l_lap_driver == last_lap_num-1:#der Fahrer ist das Rennen bis zu dieser Runde ohne DNF gefahren
                idces = race_shortened.where(race_shortened.driverId == did).dropna(how = 'all').index
                idces = list(idces)
                idx1 = idces[0]
                idx2 = idces[-1]
                race_shortened.loc[idx1:idx2, 'status_clean'] = 'No_DNF' #DNF wird als No_DNF überschrieben, da bist zu dieser Runde kein DNF stattgefunden hat
                race_shortened['status_binary'] = [1 if race_shortened.at[v,'status_clean']== 'No_DNF' else 0 for v in range(len(race_shortened.status_clean))]
        for lapnum in race_shortened.lap_number.unique():
            sum_ms = np.sum(race_shortened.where(np.logical_and(race_shortened.driverId == did,race_shortened.lap_number<=lapnum)).dropna(how = 'all')['lap_in_milliseconds'])
            
            #setzen der bisher gefahrenen Zeit (kumuliert) pro Fahrer und Runde
            race_shortened.loc[race_shortened.where(np.logical_and(race_shortened.driverId == did,race_shortened.lap_number==lapnum)).dropna(how = 'all').index,'sum_milliseconds_pro_lap'] = sum_ms
        race_shortened['sum_minutes_pro_lap'] = race_shortened['sum_milliseconds_pro_lap'] / 60000
    
    sliced_races[rid] = race_shortened
    
    

**Aufsummiere aller Boxenstops, die bis 50% des Rennens pro Fahrer gemacht wurden**

In [43]:
for key in sliced_races.keys():   #Geht alle Rennen im Dict durch
    element = sliced_races[key]
    stops = sqldf.sqldf("""
                            select *, sum(stop_binary) as sum_stops
                            from element group by driverId """)            #Summiert stops auf, pro Fahrer
    sliced_races[key] = pd.merge(sliced_races[key], stops[['driverId', 'sum_stops']], on ='driverId', how = "outer")

**Finale Auswahl der notwendigen Variablen**

In [44]:
#Rauswerfen nicht relevanter Daten und alles auf 1 Dimension bringen
for key in sliced_races.keys():
    try:
        del sliced_races[key]['status_binär']
    except:
        pass
    try:
        del sliced_races[key]['status_clean']
        del sliced_races[key]['lap_number']
        del sliced_races[key]['sum_milliseconds_pro_lap']
        del sliced_races[key]['stop_binary']
    except:
        pass
    try:
        sliced_races[key]['sum_stops'] = sliced_races[key]['sum_stops_x']
        del sliced_races[key]['sum_stops_x']
        del sliced_races[key]['sum_stops_y']
    except:
        pass

**Speichern der Erstellten Dataframes**

In [45]:
if not os.path.exists('sliced_data'):
    os.makedirs('sliced_data')


for key, value in sliced_races.items():
    name = 'sliced_data/sliced_'+str(int(key))+'.csv'
    value.to_csv(name,sep = ';', decimal = '.')


**Ende der Datenaufbereitung**